# FAST: the hydro-thermal problem

Copy-paste from https://odow.github.io/SDDP.jl/latest/examples/FAST_hydro_thermal/

We a consider a toy hydro-thermal example, adapted from https://github.com/leopoldcambier/FAST/blob/master/examples/hydro%20thermal/hydro_thermal.m

The uncertainty is the amount of rainfall at each stage. Rainfall can either be high (10) or small (2).

We can use some fuel at a price $C$ (5) to meet demand $d$ (6). The quantity of purchased fuel at stage 1 is denoted by $p_1$.

From stage to stage, water can been stored in the reservoir, but there is a tank limit V (8). We denote the stored water level at then of stage 1 by $x_1$, the used quantity at stage $1$ by $y_1$.

With these values, in the case of a two stage problem, the value function $V(x_1)$ at stage 1 is
\begin{align*}
V(x_1) &= 30 - E\left[5 \min \{ x_1 + \xi, 6 \} \right] \\ 
       &= 30 - 0.5 ( 5 \min \{ x_1+2,6 \} + 5 \min \{ x_1+10,6 \} ) \\
       &= 15 - 5/2 \min \{ x_1+2, 6 \}
\end{align*}
The problem at stage 1 is then
\begin{align*}
  \min\ & 5 p_1 + V(x_1) \\
  \text{s.t. } & x_1 \leq 8 \\
  & x_1 \leq 6 - y_1 \\
  & p_1 + y_1 \geq 6
\end{align*}
The solution at stage 1 can then trivially be found to be
$$
   x_1 = 0,\ y_1 = 6,\ p_1 = 0
$$
for an expected cost of 10.

In [5]:
using SDDP, GLPK

In [6]:
using Test

In [7]:
function fast_hydro_thermal(solver = GLPK.Optimizer)
    model = SDDP.PolicyGraph(
        SDDP.LinearGraph(2),
        bellman_function = SDDP.BellmanFunction(lower_bound = 0.0),
        optimizer = solver,
    ) do sp, t
        @variable(sp, 0 <= x <= 8, SDDP.State, initial_value = 0.0)
        @variables(sp, begin
            y >= 0
            p >= 0
            ξ
        end)
        @constraints(sp, begin
            p + y >= 6
            x.out <= x.in - y + ξ
        end)
        
        # We define the rainfall levels.
        #The level is deterministic at stage 1, but can take one of two values at subsequent stages.
        RAINFALL = (t == 1 ? [6] : [2, 10])
        
        SDDP.parameterize(sp, RAINFALL) do ω
            JuMP.fix(ξ, ω)
        end
        @stageobjective(sp, 5 * p)
    end

    det = SDDP.deterministic_equivalent(model, GLPK.Optimizer)
    JuMP.optimize!(det)
    @test JuMP.objective_value(det) == 10

    SDDP.train(model, iteration_limit = 10, log_frequency = 5)
    @test SDDP.calculate_bound(model) == 10
    return
end

fast_hydro_thermal()

------------------------------------------------------------------------------
                      SDDP.jl (c) Oscar Dowson, 2017-21

Problem
  Nodes           : 2
  State variables : 1
  Scenarios       : 2.00000e+00
  Existing cuts   : false
  Subproblem structure                      : (min, max)
    Variables                               : (6, 6)
    VariableRef in MOI.EqualTo{Float64}     : (1, 1)
    VariableRef in MOI.LessThan{Float64}    : (1, 2)
    AffExpr in MOI.LessThan{Float64}        : (1, 1)
    AffExpr in MOI.GreaterThan{Float64}     : (1, 1)
    VariableRef in MOI.GreaterThan{Float64} : (4, 4)
Options
  Solver          : serial mode
  Risk measure    : SDDP.Expectation()
  Sampling scheme : SDDP.InSampleMonteCarlo

Numerical stability report
  Non-zero Matrix range     [1e+00, 1e+00]
  Non-zero Objective range  [1e+00, 5e+00]
  Non-zero Bounds range     [8e+00, 8e+00]
  Non-zero RHS range        [6e+00, 6e+00]
No problems detected

 Iteration    Simulation       Bou